In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tweets=pd.read_csv("/kaggle/input/trainings/narendramodi_tweets.csv")
tweets.shape

In [ ]:
#Document Term Matrix¶

In [ ]:
reviews=pd.read_csv("/kaggle/input/trainings/amazon_reviews_big.csv")
reviews.shape
reviews.head()

In [ ]:
# https://jmcauley.ucsd.edu/data/amazon/


reviews = pd.read_csv('/kaggle/input/trainings/amazon_reviews_big.csv').sample(10000)
reviews['sentiment'] = reviews['overall'].apply(lambda v: 'positive' if v>=3 else 'negative')
print(reviews.shape)
reviews.head()

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer

stemmer = PorterStemmer()
docs = reviews['reviewText'].fillna('NA').str.lower().str.replace('[^a-z\s]', '')
docs = docs.apply(remove_stopwords)
docs = stemmer.stem_documents(docs)
docs = pd.Series(docs)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

train_x, test_x, train_y, test_y = train_test_split(docs, reviews['sentiment'],
                                                   test_size=0.2, random_state=1)

vectorizer = CountVectorizer().fit(train_x)

In [ ]:
vocab = vectorizer.get_feature_names()
vocab_size = len(vocab)
print('We have %d words across %d documents' % (vocab_size, train_x.shape[0]))

In [ ]:
train_dtm = vectorizer.transform(train_x)
test_dtm = vectorizer.transform(test_x)
train_dtm

In [ ]:
uncompressed_matrix = train_dtm.toarray()
df_train_dtm = pd.DataFrame(uncompressed_matrix, columns=vocab, index=train_x.index)
df_test_dtm = pd.DataFrame(test_dtm.toarray(), columns=vocab, index=test_x.index)

In [ ]:
### Columwise sum (Terms frequency)

In [ ]:
top_words = df_train_dtm.sum().sort_values(ascending=False).head(25)

In [ ]:
# Analysis on document length (i.e. no. of tokens per document)
df_train_dtm.sum(axis=1).sort_values(ascending=False).describe()

In [ ]:
# Analysis on document length (i.e. no. of tokens per document)
df_train_dtm.sum(axis=1).sort_values(ascending=False).plot.box()